In [1]:
data_var = '2023-12-24'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8748,2023-12-24,Eua Nba,00:00,Sacramento Kings,Minnesota Timberwolves,1.69,2.36,232.5,1.86,2.07,-3.5,2.05,1.89,IVwhv8bn,0.591716,0.423729,0.537634,0.483092,0.015445,0.8,0.8,NaN,NaN,196.564,39.530949,0.201110,197.058,30.079045,0.152641,187.222,196.056,197.54,204.96,0.0,0.0,0.0,0.0,0.233956,0.075569,0.057430,0.73,0.146,4.726027,0.625845,0.8,0.174155,1.13,0.226,6.017699,0.58459,0.8,0.21541
8749,2023-12-24,Itália Liga A,08:00,Milano,Cremona,1.18,4.96,158.5,1.87,1.92,-12.5,2.05,1.73,ru6Zn4Ke,0.847458,0.201613,0.534759,0.520833,0.049071,0.0,0.0,NaN,NaN,108.370,18.119510,0.167200,165.760,51.821405,0.312629,0.000,0.000,127.10,160.50,0.0,0.0,0.0,0.0,0.870640,0.018657,0.119722,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
8750,2023-12-24,Austrália Nbl,00:00,Adelaide 36ers,Brisbane Bullets,2.03,1.79,177.5,1.80,1.86,1.5,1.89,1.90,O2muN2Oa,0.492611,0.558659,0.555556,0.537634,0.051270,0.0,0.2,NaN,NaN,281.334,160.765592,0.571440,167.566,42.248285,0.252129,0.000,252.482,194.93,222.36,0.0,0.0,0.0,0.0,0.088851,0.023184,0.003731,0.00,0.000,inf,0.000000,0.0,0.000000,0.65,0.130,6.076923,0.00000,0.0,0.00000
8751,2023-12-24,Bósnia E Herzegovina Prvenstvo Bih,14:00,Prijedor,Spars Ilidza,2.32,1.56,166.5,1.85,1.85,2.5,1.99,1.73,hQJZh0Wd,0.431034,0.641026,0.540541,0.540541,0.072060,0.0,0.0,NaN,NaN,269.856,75.276721,0.278951,219.500,125.238460,0.570562,0.000,0.000,247.04,194.04,0.0,0.0,0.0,0.0,0.277011,0.000000,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
8752,2023-12-24,Bósnia E Herzegovina Prvenstvo Bih,15:00,Leotar Trebinje,Promo,1.32,3.20,161.5,1.85,1.85,-8.5,2.08,1.67,WGIwhKo3,0.757576,0.312500,0.540541,0.540541,0.070076,0.0,0.0,NaN,NaN,172.480,81.897257,0.474822,306.284,133.890684,0.437146,0.000,0.000,119.32,373.68,0.0,0.0,0.0,0.0,0.588213,0.000000,0.154621,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8824,2023-12-24,Turquia Tbl,11:30,Mersin B. B. S.,Harem Spor,1.27,3.53,149.5,1.83,1.87,-9.5,2.04,1.69,MwqYgufR,0.787402,0.283286,0.546448,0.534759,0.070688,0.0,0.0,NaN,NaN,148.214,39.056901,0.263517,137.624,65.190756,0.473687,0.000,0.000,127.05,135.24,0.0,0.0,0.0,0.0,0.665859,0.015289,0.132701,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
8825,2023-12-24,Turquia Tb2L,13:00,Cozum Ankara,Konya BBSK,3.53,1.27,136.5,1.84,1.86,7.5,1.96,1.75,tzhSwvSd,0.283286,0.787402,0.543478,0.537634,0.070688,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,86.73,114.54,0.0,0.0,0.0,0.0,0.665859,0.007644,0.080050,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
8826,2023-12-24,Turquia Kbsl Feminina,08:00,Bursa F,Bellona Kayseri F,1.86,1.86,146.5,1.83,1.87,-1.5,1.99,1.73,bDmWHqxQ,0.537634,0.537634,0.546448,0.534759,0.075269,0.0,0.0,NaN,NaN,292.324,260.572892,0.891384,243.610,186.091518,0.763891,0.000,0.000,132.80,122.24,0.0,0.0,0.0,0.0,0.000000,0.015289,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
8827,2023-12-24,Turquia Kbsl Feminina,09:00,Tarsus F,Besiktas F,3.12,1.33,152.5,1.87,1.79,5.5,2.07,1.67,j5ubD3ps,0.320513,0.751880,0.534759,0.558659,0.072393,0.0,0.0,NaN,NaN,199.504,91.937969,0.460833,186.446,152.911318,0.82013

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Sacramento Kings,Minnesota Timberwolves,232.5,1.86,1.0,Over
